<a href="https://colab.research.google.com/github/D1emonk/My-repo/blob/main/%D0%A1ompression_Techniques_Explained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.3 MB/s eta 0:00:00


In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
import torch.nn.utils.prune as prune # Обрезка весов (Pruning)
from transformers import BitsAndBytesConfig, AutoModelForCausalLM # Квантование (Quantization)
from transformers import AutoModelForSequenceClassification # Дистилляция знаний (Knowledge Distillation)
from peft import get_peft_model, LoraConfig, TaskType # LoRA (низкоранговая адаптация)

In [16]:
from transformers import AutoModel, AutoConfig

# Загружаем конфигурацию модели
config = AutoConfig.from_pretrained("bert-base-uncased")

# Выводим информацию
print(config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.54.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



<h3> Обрезка весов (Pruning)

In [10]:
# Применить L1-прореживание ко всем полносвязным слоям
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name="weight", amount=0.3)
        weight = module.weight.data
        zero_count = torch.sum(weight == 0).item()
        total = weight.numel()
        print(f"{name}: {zero_count} из {total} весов обнулено ({100 * zero_count / total:.2f}%)")

# (по желанию) удалить маски и объединить веса
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.remove(module, 'weight')

bert.encoder.layer.0.attention.self.query.base_layer: 176947 из 589824 весов обнулено (30.00%)
bert.encoder.layer.0.attention.self.query.lora_A.default: 1843 из 6144 весов обнулено (30.00%)
bert.encoder.layer.0.attention.self.query.lora_B.default: 6144 из 6144 весов обнулено (100.00%)
bert.encoder.layer.0.attention.self.key: 176947 из 589824 весов обнулено (30.00%)
bert.encoder.layer.0.attention.self.value.base_layer: 176947 из 589824 весов обнулено (30.00%)
bert.encoder.layer.0.attention.self.value.lora_A.default: 1843 из 6144 весов обнулено (30.00%)
bert.encoder.layer.0.attention.self.value.lora_B.default: 6144 из 6144 весов обнулено (100.00%)
bert.encoder.layer.0.attention.output.dense: 176947 из 589824 весов обнулено (30.00%)
bert.encoder.layer.0.intermediate.dense: 707789 из 2359296 весов обнулено (30.00%)
bert.encoder.layer.0.output.dense: 707789 из 2359296 весов обнулено (30.00%)
bert.encoder.layer.1.attention.self.query.base_layer: 176947 из 589824 весов обнулено (30.00%)
bert.

<h3> Квантование (Quantization)

In [12]:
quant_config = BitsAndBytesConfig(load_in_8bit=True)
quant_model = AutoModelForCausalLM.from_pretrained(
    "bert-base-uncased",
    quantization_config=quant_config,
    device_map="auto"
)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [13]:
for name, param in quant_model.named_parameters():
    print(name, param.dtype)
    break  # для примера, достаточно одной проверки

bert.embeddings.word_embeddings.weight torch.float16


<h3> Дистилляция знаний (Knowledge Distillation)

In [7]:
teacher = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)
student = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def count_params(model):
    return sum(p.numel() for p in model.parameters())

print("Teacher params:", count_params(teacher))
print("Student params:", count_params(student))

Teacher params: 109483778
Student params: 66955010


<h3>  LoRA (низкоранговая адаптация)

In [8]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "value"],  # адаптируем attention
    task_type=TaskType.SEQ_CLS
)

lora_model = get_peft_model(model, lora_config)

In [15]:
total_params = sum(p.numel() for p in model.parameters())
lora_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)

print(f"Всего параметров: {total_params}")
print(f"Обучаемых параметров в LoRA: {lora_params}")
print(f"Процент обучаемых: {100 * lora_params / total_params:.2f}%")

Всего параметров: 109780228
Обучаемых параметров в LoRA: 296450
Процент обучаемых: 0.27%
